In [1]:
#!pip install psycopg2

In [2]:
import pandas as pd
import sqlalchemy
import psycopg2
import matplotlib.pyplot as plt
from pprint import pprint

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine("postgresql+psycopg2://postgres:Bias1981@/EmployeeSQL")
connection = engine.connect()

In [5]:
# import the departments table
departments = pd.read_sql('select * from departments', connection)
departments

,dept_no,dept_name
0,d001,Marketing
1,d002,Finance
2,d003,Human Resources
3,d004,Production
4,d005,Development
5,d006,Quality Management
6,d007,Sales
7,d008,Research
8,d009,Customer Service


In [6]:
dept_emp = pd.read_sql('select * from dept_emp', connection)
dept_emp

,emp_no,dept_no
0,10001,d005
1,10002,d007
2,10003,d004
3,10004,d004
4,10005,d003
...,...,...
331598,499995,d004
331599,499996,d004
331600,499997,d005
331601,499998,d002


In [7]:
dept_manager = pd.read_sql('select * from dept_manager', connection)
dept_manager.head()

,dept_no,emp_no
0,d001,110022
1,d001,110039
2,d002,110085
3,d002,110114
4,d003,110183


In [9]:
employees = pd.read_sql('select * from employees', connection)
employees.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,1990-04-28
1,475053,e0002,11/18/1954,Byong,Delgrande,F,1991-09-07
2,57444,e0002,1/30/1958,Berry,Babb,F,1992-03-21
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,1987-11-26
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1991-01-18


In [10]:
salaries = pd.read_sql('select * from salaries', connection)
salaries.head()

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228


In [11]:
titles = pd.read_sql('select * from titles', connection, parse_dates=['from_date', 'to_date'])
titles

,title_id,title
0,s0001,Staff
1,s0002,Senior Staff
2,e0001,Assistant Engineer
3,e0002,Engineer
4,e0003,Senior Engineer
5,e0004,Technique Leader
6,m0001,Manager


In [12]:
employee_salary = pd.merge(employees, salaries, on='emp_no')
employee_salary.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,1990-04-28,40000
1,475053,e0002,11/18/1954,Byong,Delgrande,F,1991-09-07,53422
2,57444,e0002,1/30/1958,Berry,Babb,F,1992-03-21,48973
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,1987-11-26,40000
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1991-01-18,40000


In [13]:
titles_ren = titles.rename(columns = {"title_id":"emp_title_id"})
titles_ren

,emp_title_id,title
0,s0001,Staff
1,s0002,Senior Staff
2,e0001,Assistant Engineer
3,e0002,Engineer
4,e0003,Senior Engineer
5,e0004,Technique Leader
6,m0001,Manager


In [14]:
emp_title = pd.merge(employee_salary, titles_ren, on='emp_title_id')
emp_title.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary,title
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,1990-04-28,40000,Staff
1,421786,s0001,9/28/1957,Xiong,Verhoeff,M,1987-11-26,40000,Staff
2,273487,s0001,4/14/1957,Christoph,Parfitt,M,1991-06-28,56087,Staff
3,246449,s0001,3/23/1958,Subbu,Bultermann,F,1988-03-25,87084,Staff
4,48085,s0001,1/19/1964,Venkatesan,Gilg,M,1993-06-28,63016,Staff


In [47]:
# DELETE
emp_title.describe()

,emp_no,salary
count,300024.000000,300024.000000
mean,253321.763392,52970.732451
std,161828.235540,14301.478491
min,10001.000000,40000.000000
25%,85006.750000,40000.000000
50%,249987.500000,48681.000000
75%,424993.250000,61758.000000
max,499999.000000,129492.000000


In [45]:
# Create a histogram to visualize the most common salary ranges for employees.

In [83]:
bins = [0, 49999.999, 74999.999, 99999.999, 130000]
group_name = ['< $50K', "$50K - 75K", "$75K - 100K", ">$100K"]

emp_title['salary_bins'] = pd.cut(emp_title["salary"], bins, labels = group_name, include_lowest=True)
sal_grp = pd.DataFrame(emp_title.groupby('salary_bins')['title'].count())
sal_grp

,title
salary_bins,
< $50K,158465
$50K - 75K,113541
$75K - 100K,26730
>$100K,1288


In [23]:
emp_sal_title = emp_title[['title', 'salary']]
emp_sal_title.head()

,title,salary
0,Staff,40000
1,Staff,40000
2,Staff,56087
3,Staff,87084
4,Staff,63016


In [58]:
emp_sal_title.groupby('title')['salary'].mean().round(2)

title
Assistant Engineer    48564.43
Engineer              48535.34
Manager               51531.04
Senior Engineer       48506.80
Senior Staff          58550.17
Staff                 58465.38
Technique Leader      48582.90
Name: salary, dtype: float64